# 🎯 Unsupervised Image Descriptors Hackathon
## Classical Computer Vision Approaches on STL-10 Dataset

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/yourusername/unsupervised-descriptors-FlowersCS/blob/main/demo_notebook.ipynb)

**Team:** FlowersCS  
**Challenge:** Unsupervised image representation learning using classical descriptors  
**Dataset:** STL-10 (100k unlabeled + 5k+8k labeled images)

---

## 📋 Project Overview

This notebook demonstrates a comprehensive evaluation of classical computer vision descriptors for unsupervised image representation learning. We compare:

### 🌐 Global Descriptors
- **HOG** (Histogram of Oriented Gradients)
- **LBP** (Local Binary Patterns) 
- **Color Histograms**
- **GIST** (Global scene descriptor)

### 🔍 Local Descriptors + Encoding
- **SIFT, ORB, BRISK, SURF** + Bag of Words
- **SIFT, ORB, BRISK, SURF** + VLAD Encoding
- **SIFT, ORB, BRISK, SURF** + Fisher Vectors

### 🎯 Evaluation Protocol
- **Classification performance** (Linear SVM, Random Forest, Logistic Regression)
- **Robustness testing** (noise, blur, brightness changes)
- **Cross-validation** for reliability
- **Efficiency analysis** (speed vs accuracy)

## 🚀 Setup and Installation

Let's start by setting up the environment and downloading our project.

In [ ]:
# Install required packages
!pip install opencv-python scikit-learn scikit-image matplotlib seaborn pandas torchvision pillow

# For Google Colab - install additional dependencies
import sys
if 'google.colab' in sys.modules:
    !apt-get update
    !apt-get install -y libgl1-mesa-glx libglib2.0-0 libsm6 libxext6 libxrender-dev libgomp1

In [ ]:
# Clone the project repository (replace with your actual repository URL)
import os
if not os.path.exists('unsupervised-descriptors-FlowersCS'):
    !git clone https://github.com/yourusername/unsupervised-descriptors-FlowersCS.git
    
# Change to project directory
%cd unsupervised-descriptors-FlowersCS

# Verify project structure
!ls -la

## 📥 Data Download

Download and prepare the STL-10 dataset for our experiments.

In [ ]:
# Download STL-10 dataset
!python scripts/download_data.py

# Verify data download
!ls -la data/stl10_binary/

## 🎬 Quick Demo

Let's run a quick demonstration with a subset of data to show the complete pipeline.

In [ ]:
# Run quick test with reduced dataset
!python run_demo.py --quick-test --global-only

print("\n" + "="*60)
print("🎉 QUICK DEMO COMPLETED!")
print("="*60)

## 📊 View Results

Let's examine the results from our quick test.

In [ ]:
import json
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Load evaluation results
try:
    with open('results/evaluation_results.json', 'r') as f:
        results = json.load(f)
    
    print("✅ Results loaded successfully!")
    
    # Display configuration
    config = results.get('evaluation_config', {})
    print(f"\nEvaluation Configuration:")
    print(f"- Descriptors: {config.get('descriptors', 'N/A')}")
    print(f"- Training samples: {config.get('n_train_samples', 'N/A')}")
    print(f"- Test samples: {config.get('n_test_samples', 'N/A')}")
    print(f"- Classes: {config.get('n_classes', 'N/A')}")
    
except FileNotFoundError:
    print("❌ Results file not found. Please run the demo first.")
    results = None

In [ ]:
# Create performance summary table
if results:
    descriptor_results = results.get('descriptors', {})
    
    performance_data = []
    
    for desc_name, desc_result in descriptor_results.items():
        if desc_result.get('status') == 'success':
            classifiers = desc_result.get('classifiers', {})
            feature_info = desc_result.get('feature_info', {})
            
            for clf_name, clf_result in classifiers.items():
                if clf_result.get('status') == 'success':
                    metrics = clf_result.get('test_metrics', {})
                    performance_data.append({
                        'Descriptor': desc_name.replace('_', ' ').title(),
                        'Classifier': clf_name.replace('_', ' ').title(),
                        'Accuracy': metrics.get('accuracy', 0),
                        'F1-Score': metrics.get('macro_f1', 0),
                        'Precision': metrics.get('macro_precision', 0),
                        'Recall': metrics.get('macro_recall', 0),
                        'Dimensions': feature_info.get('dimensions', 0),
                        'Time (ms/img)': feature_info.get('avg_time_per_image', 0) * 1000
                    })
    
    if performance_data:
        df = pd.DataFrame(performance_data)
        
        # Sort by accuracy
        df = df.sort_values('Accuracy', ascending=False)
        
        print("🏆 Performance Summary (Top Results):")
        print("=" * 80)
        
        # Display top 10 results
        display_df = df.head(10).round(3)
        print(display_df.to_string(index=False))
        
        # Best performer
        best = df.iloc[0]
        print(f"\n🥇 Best Performer: {best['Descriptor']} + {best['Classifier']}")
        print(f"   Accuracy: {best['Accuracy']:.3f}")
        print(f"   F1-Score: {best['F1-Score']:.3f}")
        print(f"   Dimensions: {int(best['Dimensions'])}")
        print(f"   Speed: {best['Time (ms/img)']:.2f} ms/image")
    else:
        print("❌ No performance data found.")
else:
    print("❌ No results to display.")

In [ ]:
# Create performance visualization
if results and 'performance_data' in locals():
    # Performance comparison plot
    plt.figure(figsize=(14, 8))
    
    # Get best accuracy for each descriptor
    best_per_descriptor = df.groupby('Descriptor')['Accuracy'].max().sort_values(ascending=True)
    
    # Create horizontal bar plot
    bars = plt.barh(range(len(best_per_descriptor)), best_per_descriptor.values, 
                    color=plt.cm.viridis(np.linspace(0, 1, len(best_per_descriptor))))
    
    plt.yticks(range(len(best_per_descriptor)), best_per_descriptor.index)
    plt.xlabel('Accuracy', fontsize=12)
    plt.title('Descriptor Performance Comparison\n(Best Accuracy per Descriptor)', 
              fontsize=14, fontweight='bold')
    plt.grid(axis='x', alpha=0.3)
    
    # Add value labels
    for i, bar in enumerate(bars):
        width = bar.get_width()
        plt.text(width + 0.005, bar.get_y() + bar.get_height()/2, 
                f'{width:.3f}', ha='left', va='center', fontweight='bold')
    
    plt.tight_layout()
    plt.show()
    
    # Efficiency vs Accuracy scatter plot
    plt.figure(figsize=(12, 8))
    
    # Group by descriptor and get best results
    best_results = df.loc[df.groupby('Descriptor')['Accuracy'].idxmax()]
    
    scatter = plt.scatter(best_results['Time (ms/img)'], best_results['Accuracy'], 
                         s=best_results['Dimensions']/20, alpha=0.7,
                         c=range(len(best_results)), cmap='viridis')
    
    # Add labels
    for i, row in best_results.iterrows():
        plt.annotate(row['Descriptor'], 
                    (row['Time (ms/img)'], row['Accuracy']),
                    xytext=(5, 5), textcoords='offset points',
                    fontsize=10, alpha=0.8)
    
    plt.xlabel('Time per Image (ms)', fontsize=12)
    plt.ylabel('Accuracy', fontsize=12)
    plt.title('Efficiency Analysis: Accuracy vs Speed\n(Bubble size = Feature Dimensions)', 
              fontsize=14, fontweight='bold')
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
else:
    print("❌ No data available for visualization.")

## 🎯 Full Evaluation (Optional)

Run the complete evaluation with all descriptors (this will take longer).

In [ ]:
# Uncomment and run for full evaluation
# Warning: This will take 15-30 minutes depending on hardware

# !python run_demo.py --max-samples 5000

print("To run full evaluation, uncomment the line above.")
print("This will train and evaluate all descriptors with 5000 samples.")
print("Expected runtime: 15-30 minutes")

## 🖼️ Sample Images and Features

Let's visualize some sample images from the STL-10 dataset and their extracted features.

In [ ]:
import sys
import numpy as np
sys.path.append('src')

try:
    from utils.data_loader import STL10Dataset
    from utils.preprocessing import ImagePreprocessor
    
    # Load sample data
    dataset = STL10Dataset('data')
    X_test, y_test = dataset.load_test_data()
    
    # Class names
    class_names = ['airplane', 'bird', 'car', 'cat', 'deer', 
                   'dog', 'horse', 'monkey', 'ship', 'truck']
    
    print(f"✅ Loaded {len(X_test)} test images")
    print(f"Image shape: {X_test.shape[1:]}")
    print(f"Classes: {class_names}")
    
except Exception as e:
    print(f"❌ Error loading data: {e}")
    X_test, y_test = None, None

In [ ]:
# Visualize sample images
if X_test is not None:
    # Select one image per class
    fig, axes = plt.subplots(2, 5, figsize=(15, 6))
    axes = axes.ravel()
    
    for i, class_idx in enumerate(range(10)):
        # Find first image of this class
        class_mask = y_test == class_idx
        if np.any(class_mask):
            img_idx = np.where(class_mask)[0][0]
            img = X_test[img_idx]
            
            axes[i].imshow(img)
            axes[i].set_title(f'{class_names[class_idx]}\n(Class {class_idx})', fontsize=10)
            axes[i].axis('off')
        else:
            axes[i].text(0.5, 0.5, 'No samples', ha='center', va='center', 
                        transform=axes[i].transAxes)
            axes[i].set_title(f'{class_names[class_idx]}\n(No samples)', fontsize=10)
            axes[i].axis('off')
    
    plt.suptitle('STL-10 Dataset Sample Images (One per Class)', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()
    
    # Show class distribution
    unique, counts = np.unique(y_test, return_counts=True)
    
    plt.figure(figsize=(12, 6))
    bars = plt.bar([class_names[i] for i in unique], counts, 
                   color=plt.cm.tab10(np.linspace(0, 1, len(unique))))
    
    plt.title('Class Distribution in Test Set', fontsize=14, fontweight='bold')
    plt.xlabel('Classes', fontsize=12)
    plt.ylabel('Number of Images', fontsize=12)
    plt.xticks(rotation=45)
    
    # Add value labels on bars
    for bar, count in zip(bars, counts):
        plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 5,
                str(count), ha='center', va='bottom', fontweight='bold')
    
    plt.tight_layout()
    plt.show()
    
else:
    print("❌ No data available for visualization.")

## 🔍 Feature Extraction Demo

Let's demonstrate feature extraction with one of our descriptors.

In [ ]:
# Demonstrate feature extraction
if X_test is not None:
    try:
        from descriptors.global_descriptors import HOGDescriptor
        
        # Initialize HOG descriptor
        hog = HOGDescriptor()
        
        # Take a small sample
        sample_images = X_test[:5]
        sample_labels = y_test[:5]
        
        # Fit and extract features
        print("Extracting HOG features...")
        hog.fit(sample_images)
        features = hog.extract(sample_images)
        
        print(f"✅ Features extracted successfully!")
        print(f"Feature matrix shape: {features.shape}")
        print(f"Feature vector length: {features.shape[1]}")
        print(f"Feature range: [{features.min():.3f}, {features.max():.3f}]")
        
        # Visualize feature distributions
        plt.figure(figsize=(15, 4))
        
        # Feature histogram
        plt.subplot(1, 3, 1)
        plt.hist(features.flatten(), bins=50, alpha=0.7, edgecolor='black')
        plt.title('HOG Feature Value Distribution')
        plt.xlabel('Feature Value')
        plt.ylabel('Frequency')
        
        # Feature vector for first image
        plt.subplot(1, 3, 2)
        plt.plot(features[0], alpha=0.7)
        plt.title(f'HOG Features for First Image\n(Class: {class_names[sample_labels[0]]})')
        plt.xlabel('Feature Index')
        plt.ylabel('Feature Value')
        
        # Feature similarity matrix
        plt.subplot(1, 3, 3)
        from sklearn.metrics.pairwise import cosine_similarity
        similarity = cosine_similarity(features)
        
        im = plt.imshow(similarity, cmap='viridis')
        plt.title('Feature Similarity Matrix\n(Cosine Similarity)')
        plt.xlabel('Image Index')
        plt.ylabel('Image Index')
        plt.colorbar(im)
        
        # Add labels
        labels = [class_names[label] for label in sample_labels]
        plt.xticks(range(len(labels)), labels, rotation=45)
        plt.yticks(range(len(labels)), labels)
        
        plt.tight_layout()
        plt.show()
        
    except Exception as e:
        print(f"❌ Feature extraction demo failed: {e}")
        
else:
    print("❌ No data available for feature extraction demo.")

## 📝 Summary and Conclusions

Based on our experiments with classical computer vision descriptors on the STL-10 dataset:

In [ ]:
# Generate final summary
print("=" * 60)
print("🎯 HACKATHON RESULTS SUMMARY")
print("=" * 60)

print("\n📊 PROJECT HIGHLIGHTS:")
print("  • Implemented 8 classical descriptors (4 global + 4 local)")
print("  • Tested 3 encoding methods for local descriptors")
print("  • Evaluated with 3 different classifiers")
print("  • Comprehensive robustness testing")
print("  • Complete automation and reproducibility")

print("\n🏆 KEY FINDINGS:")
if 'df' in locals() and len(df) > 0:
    best = df.iloc[0]
    print(f"  • Best performer: {best['Descriptor']} ({best['Accuracy']:.1%} accuracy)")
    print(f"  • Most efficient: Feature dimensions from {df['Dimensions'].min():.0f} to {df['Dimensions'].max():.0f}")
    print(f"  • Speed range: {df['Time (ms/img)'].min():.1f} - {df['Time (ms/img)'].max():.1f} ms per image")
else:
    print("  • Run full evaluation to see detailed results")

print("\n💡 TECHNICAL INSIGHTS:")
print("  • Global descriptors provide good baseline performance")
print("  • Local descriptors with encoding can achieve higher accuracy")
print("  • Trade-offs exist between accuracy, speed, and feature dimensions")
print("  • Classical methods still competitive for many vision tasks")

print("\n🚀 REPRODUCIBILITY:")
print("  • Complete code available on GitHub")
print("  • One-command execution: python run_demo.py")
print("  • Comprehensive test suite included")
print("  • Google Colab ready for easy testing")

print("\n📁 OUTPUT FILES:")
print("  • results/evaluation_results.json - Detailed metrics")
print("  • results/evaluation_report.txt - Summary report")
print("  • results/visualizations/ - Performance plots")
print("  • cache/ - Trained models for reuse")

print("\n" + "=" * 60)
print("✨ Thank you for exploring our hackathon project! ✨")
print("🔗 Full code: https://github.com/yourusername/unsupervised-descriptors-FlowersCS")
print("=" * 60)

## 🎯 Next Steps and Extensions

To extend this work, consider:

### 🔬 Advanced Techniques
- **Deep features**: Pre-trained CNN features (ResNet, VGG)
- **Hybrid approaches**: Combining classical and deep features
- **Advanced encodings**: NetVLAD, FisherNet

### 📈 Evaluation Extensions
- **Retrieval tasks**: Image similarity and search
- **Few-shot learning**: Performance with limited labels
- **Domain transfer**: Cross-dataset evaluation

### ⚡ Optimization
- **GPU acceleration**: CUDA-optimized feature extraction
- **Quantization**: Reduced precision for deployment
- **Ensemble methods**: Combining multiple descriptors

---

**Team FlowersCS** - Demonstrating that classical computer vision methods remain valuable tools in the modern ML toolkit! 🌸